In [16]:
import json
import pandas as pd
import pymysql

# Cargar el archivo JSON
with open('meteorología_2023.json') as f:
    data = json.load(f)

# Aplanar la estructura del JSON (con manejo de valores faltantes)
flattened_data = []
for date, measurements in data['pollutionMeasurements']['date'].items():
    for station, values in measurements.items():
        flattened_data.append([
            date,
            station,
            values.get('RH', None),
            values.get('TMP', None),
            values.get('WDR', None),
            values.get('WSP', None),
            values.get('PBa', None)
        ])

# Crear DataFrame con nombres de columnas explícitos
df = pd.DataFrame(flattened_data, columns=['Fecha y Hora', 'Estación', 'RH', 'TMP', 'WDR', 'WSP', 'PBa'])

# Corregir la hora "24:00" (si existe) a "00:00"
df['Fecha y Hora'] = df['Fecha y Hora'].astype(str).str.replace(' 24:00', ' 00:00', regex=False)

# Convertir la columna 'Fecha y Hora' a datetime y luego a cadena de texto
df['Fecha y Hora'] = pd.to_datetime(df['Fecha y Hora']).dt.strftime('%Y-%m-%d %H:%M:%S')

# Conectar a MySQL (agregando el parámetro charset)
connection = pymysql.connect(
    host='34.72.69.10',
    user='bd_conagua',
    password='HkW0eiFxfT5iUv31',
    database='conagua',
    port=3306,
    charset='utf8mb4'  # Codificación UTF-8 para caracteres especiales
)

cursor = connection.cursor()

# Crear la tabla si no existe
create_table_query = """
CREATE TABLE IF NOT EXISTS mediciones (
    `Fecha y Hora` DATETIME,
    `Estación` VARCHAR(255),
    `RH` DECIMAL(5,2),
    `TMP` DECIMAL(5,2),
    `WDR` DECIMAL(5,2),
    `WSP` DECIMAL(5,2),
    `PBa` DECIMAL(5,2)
)
"""
cursor.execute(create_table_query)

# Insertar los datos en la tabla
insert_query = """
INSERT INTO mediciones (`Fecha y Hora`, `Estación`, `RH`, `TMP`, `WDR`, `WSP`, `PBa`)
VALUES (%s, %s, %s, %s, %s, %s, %s)
"""

# Convertir el DataFrame a una lista de tuplas para la inserción, convirtiendo fechas a cadenas y formateando otros tipos de datos
data_to_insert = []
for _, row in df.fillna(0).iterrows():
    data_to_insert.append(tuple(
        [
            row["Fecha y Hora"],
            row["Estación"],
            float(row["RH"]) if row["RH"] else None,
            float(row["TMP"]) if row["TMP"] else None,
            float(row["WDR"]) if row["WDR"] else None,
            float(row["WSP"]) if row["WSP"] else None,
            float(row["PBa"]) if row["PBa"] else None
        ]
    ))

cursor.executemany(insert_query, data_to_insert)

connection.commit()
cursor.close()
connection.close()

print("Datos cargados correctamente en la base de datos.")


C:\Users\danws\AppData\Local\Temp\ipykernel_30028\2266596928.py:66: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  for _, row in df.fillna(0).iterrows():


Datos cargados correctamente en la base de datos.
